In [1]:


# Importing required libraries
import pandas as pd
import re

# file paths
car_golf_dataset = pd.read_csv(r'D:\Salman work folder\ImumAi_Data_Analyst_Test\car-golf-dataset.csv', delimiter=';')
car_golf_info = pd.read_csv(r'D:\Salman work folder\ImumAi_Data_Analyst_Test\car-golf-info.csv', delimiter=';')

# Confirmation of the data loaded correctly
print("Car Golf Dataset:")
print(car_golf_dataset.head(), "\n")
print("Car Golf Info Dataset:")
print(car_golf_info.head())


Car Golf Dataset:
   source_id                                              title  \
0  286354023  Volkswagen Golf -Black Weeks-Fahrzeug, täglich...   
1  312826845  Volkswagen Golf Var. VII 2.0 TDI DSG GTD Sport...   
2  311592214  Volkswagen Golf VII Sportsvan Van/Lounge 2.0 T...   
3  312569111  Volkswagen Golf  Sportsvan 1.2 TSI Lounge AAC ...   
4  299066532  Volkswagen Golf 7 Sportsvan Allstar 1.6 TDI BM...   

        category        make model  power color  capacity  \
0     Estate Car  volkswagen  golf    123   NaN      1395   
1     Estate Car  volkswagen  golf    181   NaN      1968   
2         Saloon  volkswagen  golf    148   NaN      1968   
3  Van / Minibus  volkswagen  golf    109   NaN      1197   
4         Saloon  volkswagen  golf    109   NaN      1598   

             transmission    fuel           construction_year  price_net  \
0          Manual gearbox  Petrol  2015-01-01 00:00:00.000000    10466.0   
1  Automatic transmission  Diesel  2017-01-01 00:00:00.00000

In [2]:
#Extraction and Validation of the Engine Type
#This step extracts the engine type from the title column in car-golf-dataset and validates it against parsed_engine_type in car-golf-info.

# Step 1: Extracting Accurate Engine Type
def extract_engine_type(title, catalog_engine_types):
    """
    Extract the engine type from the title based on catalog engine types.
    """
    for engine_type in catalog_engine_types:
        if re.search(rf"\b{engine_type}\b", title, re.IGNORECASE):
            return engine_type
    return None

# Geting unique engine types from catalog
catalog_engine_types = car_golf_info['parsed_engine_type'].dropna().unique()

# Adding a new column for extracted engine type in car-golf-dataset
car_golf_dataset['extracted_engine_type'] = car_golf_dataset['title'].apply(
    lambda x: extract_engine_type(x, catalog_engine_types)
)

# Validating extracted engine types against catalog
car_golf_dataset['engine_type_validated'] = car_golf_dataset['extracted_engine_type'].apply(
    lambda x: x if x in catalog_engine_types else None
)

# Printing validation summary
print("\nEngine Type Extraction and Validation Summary:")
print(car_golf_dataset['engine_type_validated'].value_counts())




Engine Type Extraction and Validation Summary:
TSI       24999
TDI       18481
eTSI       5492
R          4681
GTI        3690
GTD        1258
e-Golf      696
Name: engine_type_validated, dtype: int64


In [4]:
#Correction and Standardizing Category
#This step matches category in car-golf-dataset to parsed_category from car-golf-info.
# Step 2: Match and Assign of Accurate Category
def match_category_from_catalog(row, catalog):
    """
    Match and assign a category to a row based on catalog data.
    """
    matches = catalog[
        (catalog['parsed_model'].str.contains(row['model'], na=False, case=False)) &
        (catalog['capacity'] == row['capacity'])
    ]
    if not matches.empty:
        return matches.iloc[0]['parsed_category']
    return None

# Adding a new column for corrected categories
car_golf_dataset['corrected_category'] = car_golf_dataset.apply(
    lambda row: match_category_from_catalog(row, car_golf_info), axis=1
)

# Printing category correction summary
print("\nCategory Correction Summary:")
print(car_golf_dataset['corrected_category'].value_counts())



Category Correction Summary:
Small Car             59421
Cabriolet/Roadster    17449
Name: corrected_category, dtype: int64


In [5]:
##Finalizing and Flagging Issues


# Step 3: Replacing Original Columns and Flagging Missing Data
car_golf_dataset['engine_type'] = car_golf_dataset['engine_type_validated']
car_golf_dataset['category'] = car_golf_dataset['corrected_category']

# Flagging rows with missing engine_type or category
car_golf_dataset['engine_type_missing'] = car_golf_dataset['engine_type'].isnull()
car_golf_dataset['category_missing'] = car_golf_dataset['category'].isnull()

# Printing summary of missing data
print("\nMissing Engine Type Rows:")
print(car_golf_dataset['engine_type_missing'].value_counts())
print("\nMissing Category Rows:")
print(car_golf_dataset['category_missing'].value_counts())





Missing Engine Type Rows:
False    59297
True     18447
Name: engine_type_missing, dtype: int64

Missing Category Rows:
False    76870
True       874
Name: category_missing, dtype: int64


In [6]:
#Saving Cleaned Dataset


# Saving cleaned dataset to a new CSV file
car_golf_dataset.to_csv(r'D:\Salman work folder\ImumAi_Data_Analyst_Test\cleaned_car_golf_dataset.csv', index=False)

print("\nCleaned dataset has been saved to 'cleaned_car_golf_dataset.csv'")



Cleaned dataset has been saved to 'cleaned_car_golf_dataset.csv'
